# fast estimation of particle properties
Tim Tyree<br>
9.29.2021<br>
`conda activate pyenv_ub`

__GOAL__: efficiently generate an inline rendering of dRdt versus 1/R and MSD versus tau

In [1]:
from lib.my_initialization import *
from lib import *
# #routines for radial neighbor velocities
# from lib.routines.compute_dRdt_in_bulk import *
# from lib.routines import compute_dRdt_in_bulk
# from lib.utils.utils_traj import get_DT
    
#magic    
%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart


In [2]:
import cupy as cp, cudf, numba.cuda as cuda
import rmm
# Switch to RMM allocator
cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

In [3]:
darkmode=False
if darkmode:
    # For darkmode plots
    from jupyterthemes import jtplot
    jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [4]:
# def unwrapper_pbc(incol, outcol, jump_thresh, width):
#     '''
#     Example Usage:
#     df['incol']=df['x']
#     grouped = df.groupby(pid_col)
#     df = grouped.apply_grouped(unwrapper_pbc,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), jump_thresh=width/2)
#     df['dx_unwrap']=df['outcol']
#     df.head()
#     '''
#     e=incol
#     de_unwrap=outcol
#     for i in range(cuda.threadIdx.x, len(e), cuda.blockDim.x):
#         de_unwrap[i]=0
#         if i>0:
#             de=e[i]-e[i-1]
#             jump_plus=de<-jump_thresh
#             jump_minus=de>jump_thresh
#             if jump_plus:
#                 de_unwrap[i]=width
#             elif jump_minus:
#                 de_unwrap[i]=-width

# def rolling_avg(incol, outcol, win_size):
#     e=incol
#     rolling_avg_e=outcol
#     for i in range(cuda.threadIdx.x, len(e), cuda.blockDim.x):
#         if i < win_size - 1:
#             # if there is not enough data to fill the window, take the average to be nan
#             rolling_avg_e[i] = np.nan
#         else:
#             total = 0
#             for j in range(i - win_size + 1, i + 1):
#                 total += e[j]
#             rolling_avg_e[i] = total / win_size

# def rolling_diff(incol, outcol, win_size=2):
#     e=incol
#     rolling_diff_e=outcol
#     for i in range(cuda.threadIdx.x, len(e), cuda.blockDim.x):
#         if i < win_size - 1:
#             # if there is not enough data to fill the window, take the average to be nan
#             rolling_diff_e[i] = np.nan
#         else:
#             j=i - win_size + 1
#             rolling_diff_e[i] = e[i]-e[j]

# def update_smoothed_trajectories(df):
#     df['event_id']= event_id_int + df[pid_col] / (1.+df[pid_col].max())
#     #compute unwrapped coordinates
#     df['x_unwrap']=df['x']+df['dx_unwrap']
#     df['y_unwrap']=df['y']+df['dy_unwrap']
#     #apply rolling difference to x and y after unwrapping
#     df['incol']=df['x_unwrap']
#     grouped = df.groupby(pid_col)
#     mdwargs={'win_size':2}
#     df = grouped.apply_grouped(rolling_diff,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                mdwargs)
#     df['diffx_unwrap']=df['outcol']

#     df['incol']=df['y_unwrap']
#     grouped = df.groupby(pid_col)
#     df = grouped.apply_grouped(rolling_diff,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                mdwargs)
#     df['diffy_unwrap']=df['outcol']
#     #drop data that isn't needed anymore
#     df.drop(columns=['incol','outcol'],inplace=True)
#     #compute the naive speed of the unwrapped trajectories in pixels per frame
#     df['speed']=cp.sqrt(df['diffx_unwrap']**2+df['diffy_unwrap']**2)#pixels per frame
#     return df

# def apply_unwrap_xy_trajectories_pbc(df,t_col,pid_col,width,height,**kwargs):
#     #now we only have good data...  we can compute moving averages for each particle!
#     #allocate memory
#     df['dx_unwrap']=0.*df['x']
#     df['dy_unwrap']=0.*df['y']
#     df['x_unwrap']=df['x']
#     df['y_unwrap']=df['y']
#     #TODO(optional): reset the index... not needed and ruins reconstruction of dropped columns at the end...  don't do it...
#     # df.reset_index(inplace=True)
#     #apply unwrapping to x and y
#     df['incol']=df['x']
#     grouped = df.groupby(pid_col)
#     uwargs={'jump_thresh':width/2,"width":width}
#     df = grouped.apply_grouped(unwrapper_pbc,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                uwargs)
#     df['dx_unwrap']=df['outcol']

#     df['incol']=df['y']
#     grouped = df.groupby(pid_col)
#     uwargs={'jump_thresh':height/2,"width":height}
#     df = grouped.apply_grouped(unwrapper_pbc,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                uwargs)
#     df['dy_unwrap']=df['outcol']

#     df.drop(columns=['incol','outcol'],inplace=True)

#     #DONE: confirmed ^that was nontrivial and reasonable looking
#     # (df['dx_unwrap']!=0).any(),(df['dy_unwrap']!=0).any()
#     # df[df['dx_unwrap']!=0].head()
#     # (True,True)

#     #aggregte over jumps
#     grouped_unwrap=df.groupby(pid_col)

#     #aggregate along a given columns in grouped_unwrap
#     result=grouped_unwrap[['dx_unwrap','dy_unwrap']].cumsum()

#     #map result back onto df using reindexing ninjitsu
#     cp_col_lst=['dx_unwrap','dy_unwrap']
#     df.reset_index(inplace=True)
#     result.reset_index(inplace=True)
#     for col in cp_col_lst:
#         df[col]=result[col]
#     df.set_index('index',inplace=True)

#     #compute unwrapped coordinates
#     df['x_unwrap']=df['x']+df['dx_unwrap']
#     df['y_unwrap']=df['y']+df['dy_unwrap']
#     return df

# def apply_moving_avg_xy_trajectories(df,t_col,pid_col,x_col='x_unwrap',y_col='y_unwrap',**kwargs):
#     diffx_col='diff'+x_col
#     diffy_col='diff'+y_col
#     #apply smoothing to x and y after unwrapping
#     df['incol']=df[x_col]
#     grouped = df.groupby(pid_col)
#     mawargs={'win_size':navg1}
#     df = grouped.apply_grouped(rolling_avg,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                mawargs)
#     df[x_col]=df['outcol']

#     df['incol']=df[y_col]
#     grouped = df.groupby(pid_col)
#     df = grouped.apply_grouped(rolling_avg,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                mawargs)
#     df[y_col]=df['outcol']

#     # #drop data that isn't needed anymore  
#     #DONE: verified that dropping data here doesn't affect the number of final nonnan values
#     # df.drop(columns=['incol','outcol'],inplace=True)
#     df.dropna(inplace=True)
#     # df.head()

#     #apply smoothing to x and y after unwrapping
#     df['incol']=df[x_col]
#     grouped = df.groupby(pid_col)
#     mdwargs={'win_size':2}
#     df = grouped.apply_grouped(rolling_diff,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                mdwargs)
#     df[diffx_col]=df['outcol']

#     df['incol']=df[y_col]
#     grouped = df.groupby(pid_col)
#     df = grouped.apply_grouped(rolling_diff,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                mdwargs)
#     df[diffy_col]=df['outcol']

#     #drop data that isn't needed anymore
#     df.drop(columns=['incol','outcol'],inplace=True)
#     # df.dropna(inplace=True)

#     #compute the naive speed of the unwrapped trajectories in pixels per frame
#     df['speed']=cp.sqrt(df[diffx_col]**2+df[diffy_col]**2)#pixels per frame
#     # df['speed']=cp.sqrt(df['diffx_unwrap']**2+df['diffy_unwrap']**2)#pixels per frame#*DS/DT*10**3 #cm/s

#     # #DONE: test and verify that the largest stepsize in the unwrapped xy is reasonable for both x and y
#     # max_speed_values=df.groupby(pid_col)['speed'].max().values
#     # plt.hist(max_speed_values.get(),bins=30)
#     # plt.xlabel('max pixel displacement between two frames')
#     # max_speed_warning=20 #pixels per frame
#     # assert ((max_speed_values.get()<max_speed_warning).all())
#     # plt.show()
#     return df

# TODO: computing dRdt versus 1/R for unconstrained random samples from smoothed and validated trajectories

In [5]:
from lib.measure.unwrap_and_smooth_cu import *
from lib.routines.unwrap_and_smooth_trajectories_cu import *

In [6]:
#DONE(dev smoothing with pbc): dev method that unwraps, keeps track of the unwrapping map, smooths the unwrapped by moving average, and then rewraps 
#DONE: unwrap each trajectory
#DONE: compute simple moving average of the x and y coordinates
#DONE: rewrap to smoothed coordinates to the original coordinate system
#DONE: clean one example input_fn
#DONE: smooth one example input_fn with pbc

In [7]:
# def return_moving_average_of_pbc_trajectories(input_fn, tavg1, pid_col, t_col, DT, 
#                                               width=200,
#                                               height=200,
#                                               use_drop_shorter_than=True,
#                                               drop_shorter_than=50,
#                                               tmin=100.,
#                                               printing=False,
#                                               **kwargs):
#     '''returns cudf dataframe that results from the routine.
#     routine reads input_fn, reversably unwraps trajectories, computes the moving average of x and y, rewraps smoothed coordinates to original coordinates (might not still obey explicit pbc of shape (0,width,0,height))
#     tavg1 is the moving average window, and drop_shorter_than is the minimum duration of trajectories to consider in the same units as the time column, t_col.  dt is the time between two frames.  '''
#     navg1=int(tavg1/DT)
#     height=width
#     df=cudf.read_csv(input_fn)
#     col_keep_lst=['x','y',pid_col,t_col]
#     #(optional) drop all columns that are not immediately relevant
#     col_drop_lst=set(df.columns).difference(col_keep_lst)
#     df.drop(columns=col_drop_lst,inplace=True)
#     if printing:
#         print(f"features dropped: {col_drop_lst}")
#         print(f"note: ^these can be recovered using the index")
#     #sort by particle and then by time 
#     df=df.sort_values([pid_col, t_col], ascending=True).copy()

#     #drop trials that are too brief... also drop rows that occur before tmin
#     if use_drop_shorter_than:
#         grouped = df.groupby(pid_col)
#         dft=grouped[t_col]
#         dfkeep=(dft.max()-dft.min() >= drop_shorter_than).copy()
#         if printing:
#             print(f"the percent of observations dropped for being briefer than {drop_shorter_than} ms is {(1-dfkeep[dfkeep].size/dfkeep.size)*100:.2f}%")
#         pid_keep_values=dfkeep[dfkeep].index.values
#         pid_keep_lst=list(pid_keep_values.get())
#         #batch the query, because f-string parsing has a limit at ~1000 list items and for efficient scalability
#         batchsize=500
#         num_batches=int(np.ceil(len(pid_keep_lst)/batchsize))
#         dfq_lst=[]
#         for j in range(num_batches):
#             dfq=df.query(f"{pid_col} in {pid_keep_lst[batchsize*j:batchsize*(j+1)]} and t >= {tmin}").copy()
#             dfq_lst.append(dfq)
#         df=cudf.concat(dfq_lst)
#         del dfq_lst, pid_keep_values, dfkeep, grouped

#     #the majority of the routine
#     apply_unwrap_xy_trajectories_pbc(df,t_col,pid_col,width,height)
#     apply_moving_avg_xy_trajectories(df,t_col,pid_col,navg1,x_col='x_unwrap',y_col='y_unwrap')
    
#     #compute rewrapped coordinates
#     df['x']=df['x_unwrap']-df['dx_unwrap']
#     df['y']=df['y_unwrap']-df['dy_unwrap']
#     #CONFIRMED: by increasing navg1, I can decrease the max displacement for all particles.

#     #add unique identifier for whole trial that is unique accross different csv files
#     #add unique identifier for each particle that is unique accross different csv files
#     fn = os.path.basename(input_fn)
#     event_id_int=float('1'+(''.join(re.findall(r'-?\d+\d*',fn))))
#     df['event_id_int']= int(event_id_int)
#     df['event_id']= event_id_int + df[pid_col] / (1.+df[pid_col].max())

#     col_keep_lst=['x','y',t_col,pid_col,'event_id_int',"dx_unwrap","dy_unwrap"]
#     dff=df[col_keep_lst].copy()

#     #DONE: drop anycolumns that are recomputed with a one liner, such as x_unwrap and diffx_unwrap, and speed
#     #DONE: collect recomputation of ^those one liners into a compact update_smoothed_trajectories function
#     # # #DONE: verified that update_smoothed_trajectories reproduces the full df to machine precision.
#     # # df_recalled=update_smoothed_trajectories(dff)
#     # # df_recalled.head()
#     # #DONE: test that all columns are contained in the dataframe reconstructed from the output dataframe
#     # assert((np.sort(df.columns.values)==np.sort(df_recalled.columns.values)).all())
#     return dff

# def return_moving_average_of_pbc_trajectories_and_save(
#         input_fn, tavg1, pid_col, t_col, DT, width, height,
#         use_drop_shorter_than, drop_shorter_than, tmin, printing):
#     dff = return_moving_average_of_pbc_trajectories(
#         input_fn,
#         tavg1,
#         pid_col,
#         t_col,
#         DT,
#         width=width,
#         height=height,
#         use_drop_shorter_than=use_drop_shorter_than,
#         drop_shorter_than=drop_shorter_than,
#         tmin=tmin,
#         printing=printing)

#     #save as csv in new folder
#     save_folder_ext = f'smoothed_trajectories_navg_{navg1}'
#     save_folder = os.path.join(os.path.dirname(os.path.dirname(input_fn)),
#                                save_folder_ext)
#     if not os.path.exists(save_folder):
#         os.mkdir(save_folder)

#     ext_str = f'_smoothed'
#     save_fn = os.path.basename(input_fn).replace('.csv', ext_str + '.csv')

#     save_dir = os.path.join(save_folder, save_fn)

#     dff.reset_index(inplace=True)
#     dff.to_csv(save_dir,index=False)
#     if printing:
#         if os.path.exists(save_dir):
#             print(f"the results were successfully saved to {save_folder}")
#         else:
#             print(f"the results were unsuccessfully saved to {save_folder}")
#     return save_dir

# def update_smoothed_trajectories(df):
#     event_id_int=df['event_id_int']
#     df['event_id']= event_id_int + df[pid_col] / (1.+df[pid_col].max())
#     #compute unwrapped coordinates
#     df['x_unwrap']=df['x']+df['dx_unwrap']
#     df['y_unwrap']=df['y']+df['dy_unwrap']
#     #apply rolling difference to x and y after unwrapping
#     df['incol']=df['x_unwrap']
#     grouped = df.groupby(pid_col)
#     mdwargs={'win_size':2}
#     df = grouped.apply_grouped(rolling_diff,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                mdwargs)
#     df['diffx_unwrap']=df['outcol']

#     df['incol']=df['y_unwrap']
#     grouped = df.groupby(pid_col)
#     df = grouped.apply_grouped(rolling_diff,
#                                    incols=['incol'],
#                                    outcols=dict(outcol=np.float64), kwargs=
#                                mdwargs)
#     df['diffy_unwrap']=df['outcol']
#     #drop data that isn't needed anymore
#     df.drop(columns=['incol','outcol'],inplace=True)
#     #compute the naive speed of the unwrapped trajectories in pixels per frame
#     df['speed']=cp.sqrt(df['diffx_unwrap']**2+df['diffy_unwrap']**2)#pixels per frame
#     return df

# def load_smoothed_trajectories(input_fn):
#     df=cudf.read_csv(input_fn)
#     update_smoothed_trajectories(df)
#     return df

In [19]:
# input_fn=search_for_file()

# #token FK
input_fn="/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-fk-200x200/param_set_8_ds_5.0_tmax_10_diffCoef_0.0005/Log/ic200x200.0.2_traj_sr_400_mem_0.csv"

#LR at DT=0.5
# input_fn=f"/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.5/trajectories/ic002.31_traj_sr_600_mem_0.csv"
# input_fn=f"/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.5/trajectories/ic002.11_traj_sr_600_mem_0.csv"

dt=0.5 #ms
ds=5 #cm
width=200
navg1=3
DS=ds/width
DT=dt
height=width
use_drop_shorter_than=True
drop_shorter_than=50 #ms
tmin=100.#ms
pid_col='particle'
t_col='t'
printing=True
tavg1=5 #moving average window, in ms
navg1=int(tavg1/DT)
save_dir=return_moving_average_of_pbc_trajectories_and_save(
        input_fn, tavg1, pid_col, t_col, DT, width, height,
        use_drop_shorter_than, drop_shorter_than, tmin, printing)


features dropped: {'grad_ux', 's', 'grad_vy', 'grad_uy', 'theta', 'f', 'v', 'dfdt', 'frame', 'n', 'dvdt', 'dsdt', 'grad_vx'}
note: ^these can be recovered using the index
the percent of observations dropped for being briefer than 50 ms is 49.45%
the results were successfully saved to /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-fk-200x200/param_set_8_ds_5.0_tmax_10_diffCoef_0.0005/smoothed_trajectories_navg_10


In [9]:
df=load_smoothed_trajectories(save_dir,pid_col,t_col)
df.head()

,index,x,y,t,particle,event_id_int,dx_unwrap,dy_unwrap,event_id,x_unwrap,y_unwrap,incol
0,3509,3.94061,112.10999,100.0,0,1200200024000,0.0,0.0,1.200200e+12,3.94061,112.10999,3.94061
1,3522,4.02528,112.15680,100.4,0,1200200024000,0.0,0.0,1.200200e+12,4.02528,112.15680,4.02528
2,3529,4.10498,112.18904,100.8,0,1200200024000,0.0,0.0,1.200200e+12,4.10498,112.18904,4.10498
3,3533,4.18598,112.22029,101.2,0,1200200024000,0.0,0.0,1.200200e+12,4.18598,112.22029,4.18598
4,3540,4.26711,112.25011,101.6,0,1200200024000,0.0,0.0,1.200200e+12,4.26711,112.25011,4.26711


In [10]:
#DONE: wrap routine up to saving
#DONE: wrap routine with saving
#TODO: run ^this function over all csv files in the folder of input_fn



#for loop implementation has an expected run time of ~30 minutes for LR data with DT=0.5
#for loop implementation has an expected run time of ~6 minutes for FK data with DT=0.4

In [11]:
#

In [12]:
#compute the distance in x between any points where two spiral tips occur at the same time in t

In [13]:
#DONT: add the removed columns back to df before saving.  they can be recovered using df.index

In [14]:
# #TODO: compute squared displacements
# grouped=df.groupby(pid_col)
# #HINT: squared displacements of particles is result
#  throws AttributeError: DataFrameGroupBy object has no attribute first 
# result = (grouped[['x','y']]-grouped[['x','y']].first())**2
# grouped[['x','y']].first()


In [15]:
#TODO(IMMEDIATE FOLLOWUP GOAL): show how insensitive a is to choice in navg1 (and/or navg2 if I'm using it!)


In [16]:
#GOAL_QUESTION: does there exist a tavg1 that produces the right expected a for a single termination event??

In [17]:
#DONT: try using dask_cuda, which is for multigpu support only
#HINT: https://www.kaggle.com/beniel/03-introduction-to-dask-and-dask-cudf
##HINT: ctrl+F for  'from dask_cuda import LocalCUDACluster'

In [18]:
#TODO(later): implement a one_step method on an element in a finite element simulation
# HINT: cudf.Grouper?
# HINT: https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#grouping-with-a-grouper-specification
# HINT: https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#transformation